In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import split_scale

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix


import warnings
warnings.filterwarnings("ignore")

# Baseline Model

For the baseline, I'm just using intuition to select columns, more feature engineering will be involved in the actual process

### Wrangle Data

- Pull dataframe from telco_churn database

In [2]:
query = '''
select * from customers;
'''
churn = pd.read_sql(query, get_db_url('telco_churn'))

- Surface level look at data
- **No Nulls**
- **total_charges should be a float**

In [3]:
churn.info()
churn.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
internet_service_type_id    7043 non-null int64
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
contract_type_id            7043 non-null int64
paperless_billing           7043 non-null object
payment_type_id             7043 non-null int64
monthly_charges 

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.9,267.4,Yes


- find errors in total_charges

In [4]:
churn[['total_charges']].sort_values('total_charges').head(12)

,total_charges
2855,
3118,
3052,
5433,
1731,
4054,
2176,
2250,
945,
1906,


In [5]:
churn.replace(r'^\s*$', np.nan, regex=True, inplace=True)
churn = churn.dropna(axis=0)
churn.total_charges = churn.total_charges.astype(float)

In [6]:
churn.describe()

,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,total_charges
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.162400,32.421786,1.872582,1.688567,2.315557,64.798208,2283.300441
std,0.368844,24.545260,0.737271,0.832934,1.149523,30.085974,2266.771362
min,0.000000,1.000000,1.000000,1.000000,1.000000,18.250000,18.800000
25%,0.000000,9.000000,1.000000,1.000000,1.000000,35.587500,401.450000
50%,0.000000,29.000000,2.000000,1.000000,2.000000,70.350000,1397.475000
75%,0.000000,55.000000,2.000000,2.000000,3.000000,89.862500,3794.737500
max,1.000000,72.000000,3.000000,3.000000,4.000000,118.750000,8684.800000


- define features and target

In [7]:
target = 'churn'
features = churn.columns.tolist()
features.remove(target)
features.remove('customer_id')
print(f' target: {target}\nfeatures:{features}')

 target: churn
features:['gender', 'senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service', 'multiple_lines', 'internet_service_type_id', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract_type_id', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges']


- encode object columns

In [8]:

for i in churn.select_dtypes(object).columns.tolist():
    churn[i] = LabelEncoder().fit_transform(churn[i])

- Scale float comumns

In [9]:
train, test = split_scale.split_my_data(churn, stratify=churn.churn)
scaler, train[['monthly_charges', 'total_charges']], test[['monthly_charges', 'total_charges']] = split_scale.standard_scaler(train[['monthly_charges', 'total_charges']], test[['monthly_charges', 'total_charges']])

### Model
- just a quick decision tree

In [15]:
tree = DecisionTreeClassifier(max_depth= 5,random_state=123)

tree.fit(train[features], train[target])
predicted_churn = tree.predict(train[features])
print(classification_report(train[target], predicted_churn))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87      4130
           1       0.67      0.50      0.58      1495

    accuracy                           0.80      5625
   macro avg       0.75      0.71      0.72      5625
weighted avg       0.79      0.80      0.79      5625



In [29]:
feature_importances = {}
for i in range(len(features)):
    feature_importances[features[i]] = list(tree.feature_importances_)[i]
sorted(feature_importances.items(), key = lambda kv: kv[1], reverse = True)

[('contract_type_id', 0.522133231282088),
 ('online_security', 0.13797118458275068),
 ('tenure', 0.09412577929981143),
 ('internet_service_type_id', 0.09148632016127721),
 ('total_charges', 0.08039683457241584),
 ('monthly_charges', 0.04066231151456985),
 ('tech_support', 0.00837735812318337),
 ('online_backup', 0.008169183610034353),
 ('senior_citizen', 0.006356362698273441),
 ('payment_type_id', 0.0040956666432481505),
 ('streaming_movies', 0.003972845836448954),
 ('gender', 0.002252921675898517),
 ('partner', 0.0),
 ('dependents', 0.0),
 ('phone_service', 0.0),
 ('multiple_lines', 0.0),
 ('device_protection', 0.0),
 ('streaming_tv', 0.0),
 ('paperless_billing', 0.0)]

### Baseline Results

- Baseline accuracy of 80%

# Minimum Viable Product

### Data Aquisition
- get telco_churn data

In [31]:
query = '''
select * 
from customers as cust
join `internet_service_types` as net
on cust.`internet_service_type_id` = net.internet_service_type_id
join `contract_types` as cont
on cust.`contract_type_id` = cont.`contract_type_id`
'''
churn_df = pd.read_sql(query, get_db_url('telco_churn'))
churn_df.info()
churn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
internet_service_type_id    7043 non-null int64
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
contract_type_id            7043 non-null int64
paperless_billing           7043 non-null object
payment_type_id             7043 non-null int64
monthly_charges 

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,internet_service_type_id,internet_service_type,contract_type_id,contract_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,1,No,2,59.9,542.4,No,1,DSL,1,Month-to-month
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,1,Yes,4,69.4,571.45,No,1,DSL,1,Month-to-month
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,1,Yes,1,48.2,340.35,No,1,DSL,1,Month-to-month
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,1,Yes,1,25.1,25.1,Yes,1,DSL,1,Month-to-month
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,1,No,3,30.5,30.5,Yes,1,DSL,1,Month-to-month


### Prepare

- drop duplicates

In [47]:
#for duplicate columns
churn_df = churn_df.loc[:,~churn_df.columns.duplicated()]

#for duplicat rows
churn_df = churn_df.drop_duplicates()

- drop redundant columns

In [54]:
churn_df = (churn_df.drop('contract_type_id', axis = 1)
                    .drop('internet_service_type_id', axis = 1))

In [56]:
churn_df.columns.tolist()

['customer_id',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'tenure',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'payment_type_id',
 'monthly_charges',
 'total_charges',
 'churn',
 'internet_service_type',
 'contract_type']

- encode columns with just a few unique values

In [82]:
for i in churn_df.columns.tolist():
    if churn_df[i].nunique() <= 5:
        churn_df[i] = LabelEncoder().fit_transform(churn_df[i])

- fix tenure white spaces

In [84]:
churn_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
churn_df = churn_df.dropna(axis=0)
churn_df.total_charges = churn_df.total_charges.astype(float)

- set features and target

In [94]:
target = ['churn']
features = churn_df.columns.tolist()
features.remove(target[0])
features.remove('customer_id')
print(f'target: {target}\nfeatures: {features}')

target: ['churn']
features: ['gender', 'senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges', 'internet_service_type', 'contract_type']


In [100]:
train, test = split_scale.split_my_data(churn_df, stratify = churn_df.churn)

- get features that need to be scaled

In [103]:
scaler_features = []
for i in features:
    if churn_df[i].nunique() > 5:
        scaler_features.append(i)

In [106]:
train_scaled, test_scaled = train, test
scaler, train_scaled[scaler_features], test_scaled[scaler_features] = split_scale.standard_scaler(train[scaler_features], test[scaler_features])
train_scaled

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,internet_service_type,contract_type
5320,9614-RMGHA,1,0,1,0,1.321511,1,2,0,2,...,0,2,0,1,2,0.901366,1.609650,1,1,0
5946,4795-UXVCJ,1,0,0,0,-0.265452,1,0,1,1,...,1,1,1,0,3,-1.502341,-0.807811,0,2,1
3204,7898-PDWQE,1,0,1,0,1.565659,1,2,2,2,...,2,0,2,0,3,0.519375,1.515438,0,0,2
1826,7766-CLTIC,0,0,0,0,-0.916514,1,0,0,0,...,2,0,2,1,1,-0.129509,-0.731873,0,0,0
6696,8241-JUIQO,0,0,0,0,1.158745,1,0,1,1,...,1,1,1,0,2,-1.514017,-0.420276,0,2,1
69,0699-NDKJM,0,0,1,0,0.995980,0,1,2,2,...,0,2,2,1,2,-0.202905,0.506009,0,0,0
4481,8020-BWHYL,0,1,0,0,-0.713057,1,0,0,2,...,0,0,0,1,3,0.349231,-0.503554,1,1,0
966,1309-XGFSN,1,1,1,1,0.792523,1,2,0,2,...,0,2,2,1,0,0.534388,0.789083,0,0,1
1997,9099-FTUHS,0,0,0,0,-0.387526,1,2,2,0,...,0,0,0,0,0,-0.348028,-0.458674,0,0,0
6280,3967-VQOGC,0,0,1,1,1.402893,1,2,1,1,...,1,1,1,1,1,-1.332196,-0.268665,0,2,2


### Explore